# Descrição do Projeto
Este projeto aborda a limpeza e preparação de dados extraídos do sistema de gestão hospitalar Tasy. O objetivo é transformar dados brutos e inconsistentes em um formato utilizável para análise exploratória dos dados, como o monitoramento de tempos de entrega de medicamentos urgentes.

Os arquivos exportados do Tasy frequentemente apresentam desafios, como formatações irregulares e valores nulos. Aqui, mostramos como superar essas dificuldades utilizando Python e bibliotecas populares de ciência de dados.

# Contexto
Os dados analisados neste projeto foram extraídos do sistema Tasy e envolvem operações de farmácia hospitalar. Os principais objetivos desse projeto são:

- Automatizar a análise de tempos de entrega para agilizar a tomada de decisão.
- Aumentar a precisão nas análises eliminando inconsistências nos dados.
- Promover eficiência operacional utilizando métodos avançados de limpeza e manipulação de dados.

# 1. Importar Dados
- Para esse projeto eu utilizei dados exportados do Tasy - sistema de gestão hospitalar (os dados estão codificados, mas por precaução, mantive o nome do hospital anônimo).
- Eu tive uma dificuldade imensa com a importação e tratamento dos dados, a codificação e a disposição dos dados estavam em um formato dificil de entender, existem muitas tratativas para ajustar o formato de dados para que fosse possível realizar as primeiras análises, mas depois de muito esforço e aprendizado foi concluído.  
- Dividi os dados em 2 - Teste e Validação

In [1]:
# @title Importar bibliotecas
from datetime import date
from datetime import timedelta
from datetime import timedelta, time
import pandas as pd
import numpy as np
from IPython.display import HTML

In [2]:
# @title Primeiras tratativas e leitura dos dados

#Ajustando as colunas conforme o documento - o tasy não cria as colunas de forma que seja possível ler em python, é necessário criar manualmente.
#copiar as colunas do documento txt e criar a lista a seguir, sem essa etapa o arquivo csv não vai criar as colunas.

columns = ['Nr atendimento',	'Setor paciente',	'Setor prescricao',	'Ds local estoque',	'Cd material',	'Ds material',	'Nr sequencia',	'Dt prescricao',	'Dt entrega setor',	'Dt recebimento setor',	'Dt geracao lote',	'Ds classificacao',	'Dt ger lote',	'Hr ger lote',	'Dt entr setor',	'Hr entr setor',	'Usuario entrega']

#Ajustando os tipos de dados das colunas conforme o documento
#Para ajustar a visualização dos tipos de dados - datas precisam estar como str (será convertido depois).

tipos_dados = {
    'Nr atendimento': str,
    'Setor paciente': str,
    'Setor prescricao': str,
    'Ds local estoque': str,
    'Cd material': str,
    'Ds material': str,
    'Nr sequencia': str,
    'Dt prescricao': str,
    'Dt entrega setor': str,
    'Dt recebimento setor': str,
    'Dt geracao lote': str,
    'Ds classificacao': str,
    'Dt ger lote': str,
    'Hr ger lote': str,
    'Hr entr setor': str,
    'Dt entr setor': str,
    'Usuario entrega': str}

# Caminho para o Arquivo
# Nesse caso foi necessário colocar o tipo de dado manualmente, a engine e o encoding - sem isso é impossível ler os dados.
tempos_bruto = pd.read_csv('/content/Tempos_Simulados.csv', delimiter=',', names=columns, dtype=tipos_dados, engine='python', encoding='utf-8', skiprows=1)

In [3]:
# @title Preparo dos dados

# ajustando o formato das datas #
# Pontos de atenção #
# Usar o formato - format='%d/%m/%Y\t%H:%M:%S' para datas no formato "01/07/2024 10:16:00"
# Para ajustar a data de 0024 para 2024 - basta realizar um código de busca e substituição (fiz apenas para 'Dt ger lote') Tempos_Limpo_Urgencia['Dt ger lote'] = Tempos_Limpo_Urgencia['Dt ger lote'].str.replace('0024', '2024')

tempos_bruto['Dt prescricao'] = pd.to_datetime(tempos_bruto['Dt prescricao'], format='%d/%m/%Y\t%H:%M:%S', errors='coerce')
tempos_bruto['Dt recebimento setor'] = pd.to_datetime(tempos_bruto['Dt recebimento setor'], format='%d/%m/%Y\t%H:%M:%S', errors='coerce')
tempos_bruto['Dt geracao lote'] = pd.to_datetime(tempos_bruto['Dt geracao lote'], format='%d/%m/%Y\t%H:%M:%S', errors='coerce')
tempos_bruto['Dt entrega setor'] = pd.to_datetime(tempos_bruto['Dt entrega setor'], format='%d/%m/%Y\t%H:%M:%S', errors='coerce')

#As horas saem no formato com data (1900-01-01) é necessário criar uma nova coluna sem essa informação com a função .dt.time

tempos_bruto['Hr ger lote'] = pd.to_datetime(tempos_bruto['Hr ger lote'], format='%H:%M:%S', errors='coerce')
tempos_bruto['Hora de geração'] = tempos_bruto['Hr ger lote'].dt.time
tempos_bruto['Hr entr setor'] = pd.to_datetime(tempos_bruto['Hr entr setor'], format='%H:%M:%S', errors='coerce')
tempos_bruto['Hora de Entrega'] = tempos_bruto['Hr entr setor'].dt.time

# Limpar valores vazios - Normalmente saem valores vazios em colunas "sem lote" ou seja Nr sequencia = 0, quando não há número de lote, quer dizer que o item não foi entregue - esses dados serão retirados.
# Retirei todas as colunas desnecessárias (parte 1) - 'Dt recebimento setor', 'Dt entr setor', 'Dt ger lote','Hr ger lote','Hr entr setor'
# Para ajustar a questão

Tempos_Limpo0 = tempos_bruto.dropna(subset=['Ds classificacao'])
Tempos_Limpo = Tempos_Limpo0.drop(columns=['Dt recebimento setor', 'Dt entr setor','Hr ger lote','Hr entr setor'])

# Fazendo os cálculos base para MEDIANA #
# Antes de iniciar o calculo é necessário usar o timedelta nas horas.
Tempos_Limpo['Hora de geração'] = pd.to_timedelta(Tempos_Limpo['Hora de geração'].astype(str))
Tempos_Limpo['Hora de Entrega'] = pd.to_timedelta(Tempos_Limpo['Hora de Entrega'].astype(str))

# Criar a coluna nova com o tempo de entrega.
Tempos_Limpo['Tempo de entrega'] = (Tempos_Limpo['Hora de Entrega'] - Tempos_Limpo['Hora de geração']).apply(lambda x: str(x)[-8:])

# Converter a coluna 'Tempo de entrega' para timedelta64[ns]
Tempos_Limpo['Tempo de entrega'] = pd.to_timedelta(Tempos_Limpo['Tempo de entrega'])

## Filtrando para apenas os itens na Urgência - 'Farmácia Central' ##

#Somente o setor 'farmácia central' entrega itens urgentes, que são classificados como 'agora'

#Selecionar o valor do filtro
valor_desejado0 = 'Farmacia Central'

# Filtrar o dataframe para 'agora' e criar uma cópia
Tempos_Limpo_Urgencia0 = Tempos_Limpo[Tempos_Limpo['Ds local estoque'].isin([valor_desejado0])].copy()

## Filtrando para apenas os itens na Urgência - 'agora' ##

#Selecionar o valor do filtro
valor_desejado = 'Agora'

# Filtrar o dataframe para 'agora' e criar uma cópia
Tempos_Limpo_Urgencia = Tempos_Limpo_Urgencia0[Tempos_Limpo_Urgencia0['Ds classificacao'].isin([valor_desejado])].copy()

# Converter 'Tempo de entrega' para minutos usando .loc para garantir segurança
Tempos_Limpo_Urgencia.loc[:, 'Tempo de entrega (minutos)'] = Tempos_Limpo_Urgencia['Tempo de entrega'].dt.total_seconds() / 60

#  Remover NaN do 'Tempo de entrega (minutos)'
Tempos_Limpo_Urgencia = Tempos_Limpo_Urgencia.dropna(subset=['Tempo de entrega (minutos)'])

# Calcular a mediana em segundos
mediana_tempo_entrega_segundos = Tempos_Limpo_Urgencia['Tempo de entrega'].dt.total_seconds().median()

# Converter mediana para objeto timedelta
mediana_tempo_entrega_urgencias = timedelta(seconds=mediana_tempo_entrega_segundos)

# Extrair horas, minutos, and segundoss do objeto timedelta
hours = mediana_tempo_entrega_urgencias.seconds // 3600
minutes = (mediana_tempo_entrega_urgencias.seconds % 3600) // 60
seconds = mediana_tempo_entrega_urgencias.seconds % 60

# Filtrar por 'Hora de geração' antes das 06:00:00 ou depois 22:00:00
hora_limite_inferior = time(6, 0, 0)
hora_limite_superior = time(22, 0, 0)

# Colocar 'Hora de geração' no formato timedelta
Tempos_Limpo_Urgencia['Hora de geração'] = pd.to_timedelta(Tempos_Limpo_Urgencia['Hora de geração'].astype(str))

# Extrair a hora da geração, Substituir NaT por 0
hora_geracao_noturno = Tempos_Limpo_Urgencia['Hora de geração'].dt.components.hours.fillna(0)

# Aplicar o filtro usando as horas limite (note que noturno tem | e diurno tem &, o que garante a filtragem correta)
filtered_indices = (hora_geracao_noturno < hora_limite_inferior.hour) | (hora_geracao_noturno > hora_limite_superior.hour)

# Filtrar o dataframe usando uma mascara boleana
Tempos_Limpo_Urgencia_noturno = Tempos_Limpo_Urgencia[filtered_indices]

# Aplicar o filtro para Tempos_Limpo_Urgencia
Tempos_Limpo_Urgencia_noturno = Tempos_Limpo_Urgencia[filtered_indices]

# Calcular o tempo de entrega em segundos
mediana_tempo_entrega_segundos_noturno = Tempos_Limpo_Urgencia_noturno['Tempo de entrega'].dt.total_seconds().median()

# Converter a mediana para objeto timedelta
mediana_tempo_entrega_noturno = timedelta(seconds=mediana_tempo_entrega_segundos_noturno)

# Extrair horas, minutos, and segundoss do objeto timedelta
hours_noturno = mediana_tempo_entrega_noturno.seconds // 3600
minutes_noturno = (mediana_tempo_entrega_noturno.seconds % 3600) // 60
seconds_noturno = mediana_tempo_entrega_noturno.seconds % 60

# Filtrar por 'Hora de geração' antes das 06:00:00 ou depois 22:00:00
hora_limite_inferior = time(6, 0, 0)
hora_limite_superior = time(22, 0, 0)

# Colocar 'Hora de geração' no formato timedelta
Tempos_Limpo_Urgencia['Hora de geração'] = pd.to_timedelta(Tempos_Limpo_Urgencia['Hora de geração'].astype(str))

# Extrair a hora da geração, Substituir NaT por 0
hora_geracao_noturno = Tempos_Limpo_Urgencia['Hora de geração'].dt.components.hours.fillna(0)

# Aplicar o filtro usando as horas limite (note que noturno tem | e diurno tem &, o que garante a filtragem correta)
filtered_indices_business_hours = (hora_geracao_noturno >= hora_limite_inferior.hour) & (hora_geracao_noturno <= hora_limite_superior.hour)

# Filtrar o dataframe usando uma mascara boleana
Tempos_Limpo_Urgencia_Diurno = Tempos_Limpo_Urgencia[filtered_indices_business_hours]

# Calcular o tempo de entrega em segundos
mediana_tempo_entrega_segundos_diurno = Tempos_Limpo_Urgencia_Diurno['Tempo de entrega'].dt.total_seconds().median()

# Converter a mediana para objeto timedelta
mediana_tempo_entrega_segundos_diurno = timedelta(seconds=mediana_tempo_entrega_segundos_diurno)

# Extrair horas, minutos, and segundoss do objeto timedelta
hours_diurno = mediana_tempo_entrega_segundos_diurno.seconds // 3600
minutes_diurno = (mediana_tempo_entrega_segundos_diurno.seconds % 3600) // 60
seconds_business_diurno = mediana_tempo_entrega_segundos_diurno.seconds % 60

mediana_tempo_entrega_Urgencias = f"Mediana do Tempo de entrega de urgências - Diurno e Noturno: {hours:02}:{minutes:02}:{seconds:02}"
mediana_tempo_entrega_Urgencias_noturno = f"Mediana do Tempo de entrega (Noturno): {hours_noturno:02}:{minutes_noturno:02}:{seconds_noturno:02}"
mediana_tempo_entrega_Urgencias_diurno = f"Mediana do Tempo de entrega (Diurno): {hours_diurno:02}:{minutes_diurno:02}:{seconds_business_diurno:02}"

# Converter data 0024 para 2024
Tempos_Limpo_Urgencia['Dt ger lote'] = Tempos_Limpo_Urgencia['Dt ger lote'].str.replace('0024', '2024')

# Eliminar colunas desnecessárias (parte 2)
# Algumas partes desse código são desnecessárias, mas fizeram parte do meu aprendizado - deixei no código, pois, no futuro podem ser uteis.
Tempos_Limpo_Urgencia = Tempos_Limpo_Urgencia.drop(columns=['Nr atendimento', 'Setor paciente','Ds local estoque','Ds material', 'Nr sequencia', 'Dt prescricao', 'Dt entrega setor', 'Dt geracao lote', 'Ds classificacao', ])

# 2 - Avalie a qualidade dos dados

- Avalie a qualidade dos seus dados e faça as devidas tratativas.

In [4]:
Tempos_Limpo_Urgencia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1084 entries, 28 to 9986
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   Setor prescricao            1084 non-null   object         
 1   Cd material                 1084 non-null   object         
 2   Dt ger lote                 1084 non-null   object         
 3   Usuario entrega             1084 non-null   object         
 4   Hora de geração             1084 non-null   timedelta64[ns]
 5   Hora de Entrega             1084 non-null   timedelta64[ns]
 6   Tempo de entrega            1084 non-null   timedelta64[ns]
 7   Tempo de entrega (minutos)  1084 non-null   float64        
dtypes: float64(1), object(4), timedelta64[ns](3)
memory usage: 76.2+ KB


In [5]:
Tempos_Limpo_Urgencia.describe()

,Hora de geração,Hora de Entrega,Tempo de entrega,Tempo de entrega (minutos)
count,1084,1084,1084,1084.000000
mean,0 days 11:30:33.210332103,0 days 10:23:41.402214022,0 days 10:51:48.487084870,651.808118
std,0 days 06:59:09.591311726,0 days 06:51:50.887157439,0 days 06:57:15.344948557,417.255749
min,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0.000000
25%,0 days 05:00:00,0 days 03:00:00,0 days 04:00:00,240.000000
50%,0 days 12:00:00,0 days 09:00:00,0 days 10:00:00,600.000000
75%,0 days 18:00:00,0 days 15:00:00,0 days 18:00:00,1080.000000
max,0 days 23:00:00,0 days 21:00:00,0 days 22:00:00,1320.000000


In [6]:
Vazio = Tempos_Limpo_Urgencia.isnull().sum()
Vazio

,0
Setor prescricao,0
Cd material,0
Dt ger lote,0
Usuario entrega,0
Hora de geração,0
Hora de Entrega,0
Tempo de entrega,0
Tempo de entrega (minutos),0


In [7]:
duplicados = Tempos_Limpo_Urgencia.duplicated().sum()
duplicados

0

# Estatisticas simples

- Código deixado apenas para mostra uma análise que não seria possível sem as tratativas acima.
- Os dados são aleatórios, portanto, os dados estatísticos não são reais.

In [8]:
# @title Estatisticas simples
# Calcula o número total de entregas de urgência
total_entregas = len(Tempos_Limpo_Urgencia)

# Calcula o número de entregas que passaram de 30 minutos
entregas_acima_30_minutos = len(Tempos_Limpo_Urgencia[Tempos_Limpo_Urgencia['Tempo de entrega (minutos)'] > 30])

# Calcula a porcentagem de entregas que passaram de 30 minutos
porcentagem_acima_30_minutos = (entregas_acima_30_minutos / total_entregas) * 100

# Calcula o percentil 0.9
percentil_90 = Tempos_Limpo_Urgencia['Tempo de entrega (minutos)'].quantile(0.9)

# Calcula a quantidade de Lotes Urgentes
distinct_count = Tempos_Limpo['Nr sequencia'].nunique()

# Calcular Qual é o valor que representa o terceiro quartil + 4*IQR
Q1 = Tempos_Limpo_Urgencia['Tempo de entrega (minutos)'].quantile(0.25)
Q3 = Tempos_Limpo_Urgencia['Tempo de entrega (minutos)'].quantile(0.75)
IQR = Q3 - Q1
outlier_threshold = Q3 + 4 * IQR

# Calcular quantos % estão acima do valor Q3 + 4*IQR
percentage_above_threshold = (Tempos_Limpo_Urgencia['Tempo de entrega (minutos)'] > outlier_threshold).mean() * 100

# Calcular quantos lotes estão acima do valor Q3 + 4*IQR
percentage_above_threshold_lotes = percentage_above_threshold*distinct_count/100

# Armazenar as saídas em variáveis
mediana_com_outliers_output = f"Mediana do Tempo de entrega de urgências - Diurno e Noturno: <b>  {hours:02}:{minutes:02}:{seconds:02} </b>"
mediana_tempo_entrega_Urgencias_noturno_output = f"Mediana do Tempo de entrega (Noturno): <b> {hours_noturno:02}:{minutes_noturno:02}:{seconds_noturno:02} </b> "
mediana_tempo_entrega_Urgencias_diurno_output = f"Mediana do Tempo de entrega (Diurno): <b>  {hours_diurno:02}:{minutes_diurno:02}:{seconds_business_diurno:02} </b> "
porcentagem_output = f"A porcentagem de itens que passaram de 30 minutos é: <b> {porcentagem_acima_30_minutos:.2f}% </b> "
percentil_output = f"O percentil 0.9 (90º percentil) do tempo de entrega das urgências é: <b> {percentil_90:.2f} minutos </b> "
distinct_count_output = f'A quantidade total de Lotes Urgentes dispensados é: <b> {distinct_count} lotes </b> '
outlier_threshold_output = f'Outliers = Q3 + 4*IQR é: <b> {outlier_threshold:.2f} minutos </b> '
percentage_above_threshold_output = f'A porcentagem de outliers é: <b> {percentage_above_threshold:.2f}% </b>, que representa <b> {percentage_above_threshold_lotes:.2f} Lotes </b> '

# Criar um HTML com todos os textos em um tamanho maior (fiz isso porque os números saem muito pequenos no meu monitor - depois descobri que podia dar zoom, mas pelo menos serviu para aprender a criar esse tipo de saída)
html_output = f"""
<font size='4'>
  <p>{mediana_com_outliers_output}</p>


  <p>{mediana_tempo_entrega_Urgencias_noturno_output}</p>


  <p>{mediana_tempo_entrega_Urgencias_diurno_output}</p>


  <p>{porcentagem_output}</p>


  <p>{percentil_output}</p>


    <p>{outlier_threshold_output}</p>

    <p>{distinct_count_output}</p>

  <p>{percentage_above_threshold_output}</p>
</font>
"""

# Exibir a saída com o HTML
display(HTML(html_output))

# 3 – Conclusão

Este projeto demonstrou a importância da limpeza e tratamento de dados para a análise exploratória de dados, especialmente em um contexto hospitalar com dados extraídos do sistema Tasy. Através da aplicação de técnicas de manipulação de dados com Python e bibliotecas como Pandas, foi possível transformar dados brutos e inconsistentes em um formato utilizável para análise.

## Resultados Alcançados:

- **Dados Limpos e Consistentes:** Os dados foram limpos, corrigidos e formatados, eliminando valores nulos, inconsistências e duplicatas, garantindo a qualidade e confiabilidade para análise.
- **Análise Exploratória Facilitada:** A etapa de limpeza e tratamento permitiu a realização de análises exploratórias dos dados, como o cálculo da mediana do tempo de entrega de medicamentos urgentes, com resultados mais precisos e confiáveis.
- **Melhoria na Tomada de Decisão:** As análises realizadas com os dados limpos fornecem insights valiosos para a tomada de decisão, como a identificação de gargalos no processo de entrega de medicamentos e a otimização do tempo de entrega.
- **Automação e Eficiência:** O código desenvolvido automatiza o processo de limpeza e tratamento de dados, tornando-o mais eficiente e permitindo a análise de grandes volumes de dados de forma rápida e precisa.

Limitações:

Os dados utilizados foram simulados, portanto as estatisticas simples não são dados reais. Apenas uma demosntração,
Algumas etapas de limpeza e tratamento podem ser específicas para o relatório utilizado (personalizado) e podem precisar de adaptações para outros cenários.

Este projeto demonstra o potencial da ciência de dados para a análise e otimização de processos hospitalares. A limpeza e tratamento de dados são etapas cruciais para garantir a qualidade e confiabilidade das análises, permitindo a tomada de decisões mais assertivas e a melhoria da eficiência dos serviços de saúde. A aplicação das técnicas e conhecimentos adquiridos neste projeto pode trazer benefícios significativos para a gestão hospitalar, contribuindo para a otimização dos processos e a melhoria da qualidade do atendimento aos pacientes.